In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

root_path = 'drive/MyDrive/NBA-predictions'

Mounted at /content/drive


In [ ]:
# our data consists of all regular season games from 2013 to the end of the 2015 season
all_games = pd.read_csv(root_path +'/data/nba_games_2013_2015.csv', sep=';')

In [ ]:
all_games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22015,1610612750,MIN,Minnesota Timberwolves,21501226,2016-04-13,MIN vs. NOP,W,240,144,56,86,0.651,13,28,0.464,19,23,0.826,5,38,43,41,14,8,13,20,35.0
1,22015,1610612749,MIL,Milwaukee Bucks,21501225,2016-04-13,MIL vs. IND,L,240,92,37,87,0.425,7,23,0.304,11,13,0.846,7,36,43,23,8,3,15,15,-5.0
2,22015,1610612738,BOS,Boston Celtics,21501217,2016-04-13,BOS vs. MIA,W,240,98,37,86,0.430,5,14,0.357,19,22,0.864,10,29,39,20,7,3,7,20,10.0
3,22015,1610612747,LAL,Los Angeles Lakers,21501228,2016-04-13,LAL vs. UTA,W,239,101,41,85,0.482,6,25,0.240,13,15,0.867,8,39,47,19,6,3,13,17,5.0
4,22015,1610612739,CLE,Cleveland Cavaliers,21501220,2016-04-13,CLE vs. DET,L,265,110,46,97,0.474,7,18,0.389,11,15,0.733,8,35,43,21,4,7,10,23,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375,22013,1610612753,ORL,Orlando Magic,21300001,2013-10-29,ORL @ IND,L,241,87,36,93,0.387,9,19,0.474,6,10,0.600,13,26,39,17,10,6,17,26,-10.0
7376,22013,1610612748,MIA,Miami Heat,21300002,2013-10-29,MIA vs. CHI,W,239,107,37,72,0.514,11,20,0.550,22,29,0.759,5,35,40,26,10,7,18,21,12.0
7377,22013,1610612747,LAL,Los Angeles Lakers,21300003,2013-10-29,LAL vs. LAC,W,240,116,42,93,0.452,14,29,0.483,18,28,0.643,18,34,52,23,8,6,19,23,13.0
7378,22013,1610612741,CHI,Chicago Bulls,21300002,2013-10-29,CHI @ MIA,L,238,95,35,83,0.422,7,26,0.269,18,23,0.783,11,30,41,23,11,4,18,27,-12.0


In [ ]:
# sort games by GAME_DATE
all_games = all_games.sort_values(by=['GAME_DATE'])
all_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
7379,22013,1610612754,IND,Indiana Pacers,21300001,2013-10-29,IND vs. ORL,W,241,97,34,71,0.479,7,17,0.412,22,32,0.688,10,34,44,17,4,18,20,13,10.0
7374,22013,1610612746,LAC,Los Angeles Clippers,21300003,2013-10-29,LAC @ LAL,L,239,103,41,83,0.494,8,21,0.381,13,23,0.565,10,30,40,27,11,4,16,21,-13.0
7375,22013,1610612753,ORL,Orlando Magic,21300001,2013-10-29,ORL @ IND,L,241,87,36,93,0.387,9,19,0.474,6,10,0.600,13,26,39,17,10,6,17,26,-10.0
7378,22013,1610612741,CHI,Chicago Bulls,21300002,2013-10-29,CHI @ MIA,L,238,95,35,83,0.422,7,26,0.269,18,23,0.783,11,30,41,23,11,4,18,27,-12.0
7377,22013,1610612747,LAL,Los Angeles Lakers,21300003,2013-10-29,LAL vs. LAC,W,240,116,42,93,0.452,14,29,0.483,18,28,0.643,18,34,52,23,8,6,19,23,13.0


In [ ]:
def get_season_by_year(year=2013, df=all_games):
  '''
  year = int
    NOTE: year = 2013, 2014, 2015 (due to dataset limitations)

  df = all_games by default

  Returns dataframe with games filtered by year.
  '''
  # Note: This function should be cleaned up to allow for more dynamic input
  # This is a quick implementation

  temp_df = df.copy()
  
  # pandas allows us to use strings to filter datetimes
  # we will use August 8th as the month and day to filter by season
  min_date = str(year) + '-08-08'
  max_date = str(year+1) + '-08-08'

  temp_df = temp_df.drop(temp_df[temp_df.GAME_DATE > max_date].index)
  temp_df = temp_df.drop(temp_df[temp_df.GAME_DATE < min_date].index)
  
  return temp_df

In [ ]:
get_season_by_year(year=2015).head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
2454,22015,1610612765,DET,Detroit Pistons,21500001,2015-10-27,DET @ ATL,W,239,106,37,96,0.385,12,29,0.414,20,26,0.769,23,36,59,23,5,3,15,15,12.0
2455,22015,1610612744,GSW,Golden State Warriors,21500003,2015-10-27,GSW vs. NOP,W,241,111,41,96,0.427,9,30,0.300,20,22,0.909,21,35,56,29,8,7,20,29,16.0
2456,22015,1610612739,CLE,Cleveland Cavaliers,21500002,2015-10-27,CLE @ CHI,L,240,95,38,94,0.404,9,29,0.310,10,17,0.588,11,39,50,26,5,7,10,21,-2.0
2457,22015,1610612740,NOP,New Orleans Pelicans,21500003,2015-10-27,NOP @ GSW,L,241,95,35,83,0.422,6,18,0.333,19,27,0.704,8,25,33,21,9,3,18,26,-16.0
2458,22015,1610612741,CHI,Chicago Bulls,21500002,2015-10-27,CHI vs. CLE,W,240,97,37,87,0.425,7,19,0.368,16,23,0.696,7,40,47,13,6,10,13,22,2.0


In [ ]:
# We're going to start with a bit of pre-processing to make things easier in the future

In [ ]:
def get_home_away_team(df=all_games):
  '''
  Assumes the `df` being passed into the function has a MATCHUP column in either of the formats:
    `team1 vs. team1`
    `team1 @ team2`

    Returns dataframe with HOME_TEAM and AWAY_TEAM columns appended to the original dataframe.

  '''
  home_team = list()
  away_team = list()

  for i in range(len(df)):
    substring = '@' # use this symbol to help identify the matchup, as the format changes depending on the team

    if substring in all_games.iloc[i].MATCHUP: # if '@' is found, the SECOND team is the HOME TEAM
      #print(f'Index: {i} - Home Team: {all_games.iloc[i].MATCHUP[-3:]} - Away Team: {all_games.iloc[i].MATCHUP[:3]}') 
      home_team.append(all_games.iloc[i].MATCHUP[-3:])
      away_team.append(all_games.iloc[i].MATCHUP[:3])

    else: # if the '@' is NOT found, the FIRST team is the HOME TEAM
      #print(f'Index: {i} - Home Team: {all_games.iloc[i].MATCHUP[:3]} - Away Team: {all_games.iloc[i].MATCHUP[-3:]}')
      home_team.append(all_games.iloc[i].MATCHUP[:3])
      away_team.append(all_games.iloc[i].MATCHUP[-3:]) 

  temp_df = df.copy()
  temp_df['HOME_TEAM'] = home_team
  temp_df['AWAY_TEAM'] = away_team

  return temp_df

In [ ]:
get_home_away_team().head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HOME_TEAM,AWAY_TEAM
7379,22013,1610612754,IND,Indiana Pacers,21300001,2013-10-29,IND vs. ORL,W,241,97,34,71,0.479,7,17,0.412,22,32,0.688,10,34,44,17,4,18,20,13,10.0,IND,ORL
7374,22013,1610612746,LAC,Los Angeles Clippers,21300003,2013-10-29,LAC @ LAL,L,239,103,41,83,0.494,8,21,0.381,13,23,0.565,10,30,40,27,11,4,16,21,-13.0,LAL,LAC
7375,22013,1610612753,ORL,Orlando Magic,21300001,2013-10-29,ORL @ IND,L,241,87,36,93,0.387,9,19,0.474,6,10,0.600,13,26,39,17,10,6,17,26,-10.0,IND,ORL
7378,22013,1610612741,CHI,Chicago Bulls,21300002,2013-10-29,CHI @ MIA,L,238,95,35,83,0.422,7,26,0.269,18,23,0.783,11,30,41,23,11,4,18,27,-12.0,MIA,CHI
7377,22013,1610612747,LAL,Los Angeles Lakers,21300003,2013-10-29,LAL vs. LAC,W,240,116,42,93,0.452,14,29,0.483,18,28,0.643,18,34,52,23,8,6,19,23,13.0,LAL,LAC


In [ ]:
def set_win_or_loss(df=all_games):
  '''
  Assumes the dataframe has a 'WL' column, with a string value of either 'W' or 'L'.

  Note: This column will be used as the TARGET variable in baseline models.

  Replaces 'W' with a '1'
  Replaces 'L' with a '0'

  Returns a DataFrame.

  '''
  temp_df = df.copy()

  win_loss_dict = {
    'W' : 1,
    'L' : 0
    }
    
  temp_df['WL'] = temp_df['WL'].replace(win_loss_dict)

  return temp_df

In [ ]:
set_win_or_loss().head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
7379,22013,1610612754,IND,Indiana Pacers,21300001,2013-10-29,IND vs. ORL,1,241,97,34,71,0.479,7,17,0.412,22,32,0.688,10,34,44,17,4,18,20,13,10.0
7374,22013,1610612746,LAC,Los Angeles Clippers,21300003,2013-10-29,LAC @ LAL,0,239,103,41,83,0.494,8,21,0.381,13,23,0.565,10,30,40,27,11,4,16,21,-13.0
7375,22013,1610612753,ORL,Orlando Magic,21300001,2013-10-29,ORL @ IND,0,241,87,36,93,0.387,9,19,0.474,6,10,0.600,13,26,39,17,10,6,17,26,-10.0
7378,22013,1610612741,CHI,Chicago Bulls,21300002,2013-10-29,CHI @ MIA,0,238,95,35,83,0.422,7,26,0.269,18,23,0.783,11,30,41,23,11,4,18,27,-12.0
7377,22013,1610612747,LAL,Los Angeles Lakers,21300003,2013-10-29,LAL vs. LAC,1,240,116,42,93,0.452,14,29,0.483,18,28,0.643,18,34,52,23,8,6,19,23,13.0


In [ ]:
# Let's create a new dataframe using these functions
# Note: we are not filtering the data by season just yet, we will save this for later

df = all_games.copy()

df = get_home_away_team(df=df)
df = set_win_or_loss(df=df)

In [ ]:
df.head()

# We can see below that the WL column has been configured correctly,
# while both the HOME and AWAY teams have been set

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HOME_TEAM,AWAY_TEAM
7379,22013,1610612754,IND,Indiana Pacers,21300001,2013-10-29,IND vs. ORL,1,241,97,34,71,0.479,7,17,0.412,22,32,0.688,10,34,44,17,4,18,20,13,10.0,IND,ORL
7374,22013,1610612746,LAC,Los Angeles Clippers,21300003,2013-10-29,LAC @ LAL,0,239,103,41,83,0.494,8,21,0.381,13,23,0.565,10,30,40,27,11,4,16,21,-13.0,LAL,LAC
7375,22013,1610612753,ORL,Orlando Magic,21300001,2013-10-29,ORL @ IND,0,241,87,36,93,0.387,9,19,0.474,6,10,0.600,13,26,39,17,10,6,17,26,-10.0,IND,ORL
7378,22013,1610612741,CHI,Chicago Bulls,21300002,2013-10-29,CHI @ MIA,0,238,95,35,83,0.422,7,26,0.269,18,23,0.783,11,30,41,23,11,4,18,27,-12.0,MIA,CHI
7377,22013,1610612747,LAL,Los Angeles Lakers,21300003,2013-10-29,LAL vs. LAC,1,240,116,42,93,0.452,14,29,0.483,18,28,0.643,18,34,52,23,8,6,19,23,13.0,LAL,LAC


In [ ]:
# We're going to feature engineer some statistics for each team, based off of their recent performance
# We'll start by specifically looking at PTS, REB, AST, STL

# We are going to create functions to make this process easier, as we will have to filter by team, gameid, etc.

In [ ]:
def filter_by_team_abrev(team_abbrev, df=all_games):
  '''
  Assumes the dataframe being passed has a column named 'TEAM_ABBREVIATION'.

  Returns Dataframe filtered by `team_abbrev`.
  '''
  temp_df = df.copy()

  df_filter = (df.TEAM_ABBREVIATION == team_abbrev)
  
  return temp_df[df_filter]

In [ ]:
filter_by_team_abrev('IND', df)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HOME_TEAM,AWAY_TEAM
7379,22013,1610612754,IND,Indiana Pacers,21300001,2013-10-29,IND vs. ORL,1,241,97,34,71,0.479,7,17,0.412,22,32,0.688,10,34,44,17,4,18,20,13,10.0,IND,ORL
7373,22013,1610612754,IND,Indiana Pacers,21300011,2013-10-30,IND @ NOP,1,240,95,30,70,0.429,10,23,0.435,25,30,0.833,6,31,37,19,5,12,16,18,5.0,NOP,IND
7313,22013,1610612754,IND,Indiana Pacers,21300033,2013-11-02,IND vs. CLE,1,240,89,32,77,0.416,10,28,0.357,15,20,0.750,10,41,51,24,6,6,20,16,15.0,IND,CLE
7266,22013,1610612754,IND,Indiana Pacers,21300053,2013-11-05,IND @ DET,1,240,99,38,73,0.521,9,21,0.429,14,23,0.609,4,41,45,26,6,9,18,21,8.0,DET,IND
7247,22013,1610612754,IND,Indiana Pacers,21300061,2013-11-06,IND vs. CHI,1,240,97,36,86,0.419,7,18,0.389,18,21,0.857,17,35,52,21,9,7,14,24,17.0,IND,CHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,22015,1610612754,IND,Indiana Pacers,21501165,2016-04-06,IND vs. CLE,1,240,123,45,80,0.563,16,31,0.516,17,22,0.773,9,29,38,25,9,1,14,24,14.0,IND,CLE
88,22015,1610612754,IND,Indiana Pacers,21501183,2016-04-08,IND @ TOR,0,241,98,37,83,0.446,8,27,0.296,16,20,0.800,9,24,33,22,7,7,9,28,-13.0,TOR,IND
72,22015,1610612754,IND,Indiana Pacers,21501199,2016-04-10,IND vs. BKN,1,240,129,48,86,0.558,11,27,0.407,22,27,0.815,14,44,58,34,10,4,20,22,24.0,IND,BKN
34,22015,1610612754,IND,Indiana Pacers,21501212,2016-04-12,IND vs. NYK,1,241,102,41,80,0.513,6,19,0.316,14,20,0.700,8,33,41,26,10,3,13,10,12.0,IND,NYK
